### **Atrybuty Dynamiczne**

In [4]:
import json

with open("osconfeed-sample.json") as f:
    data = json.load(f)

data

{'Schedule': {'conferences': [{'serial': 115}],
  'events': [{'serial': 34505,
    'name': 'Why Schools Don´t Use Open Source to Teach Programming',
    'event_type': '40-minute conference session',
    'time_start': '2014-07-23 11:30:00',
    'time_stop': '2014-07-23 12:10:00',
    'venue_serial': 1462,
    'description': 'Aside from the fact that high school programming...',
    'website_url': 'http://oscon.com/oscon2014/public/schedule/detail/34505',
    'speakers': [157509],
    'categories': ['Education']}],
  'speakers': [{'serial': 157509,
    'name': 'Robert Lefkowitz',
    'photo': None,
    'url': 'http://sharewave.com/',
    'position': 'CTO',
    'affiliation': 'Sharewave',
    'twitter': 'sharewaveteam',
    'bio': 'Robert ´r0ml´ Lefkowitz is the CTO at Sharewave, a startup...'}],
  'venues': [{'serial': 1462,
    'name': 'F151',
    'category': 'Conference Venues'}]}}

In [5]:
from collections import abc


class FrozenJSON:
    """Fasada tylko do odczytu dla przeglądania obiektu podobnego do JSON
    przy używaniu notacji atrubytów."""

    def __init__(self, mapping):
        self.__data = dict(mapping)

    def __getattr__(self, name):
        try:
            # Metoda `__getattr__` jest wywoływana tylko wtedy
            # gdy nie ma atrybutu o nazwie `name`. 
            # Jeśli `name` odpowiada instancji `__data` słownika,
            # to zwracamy wartość tego atrybutu. Tak właśnie są obługiwane
            # takie wywołania jak `.keys()`, `.items()`, `.values()`.
            # Są to metody instancji słownika.
            return getattr(self.__data, name)
        except AttributeError:
            return FrozenJSON.build(self.__data[name])

    def __dir__(self):
        return self.__data.keys()

    @classmethod
    def build(cls, obj):
        if isinstance(obj, abc.Mapping):
            return cls(obj)
        if isinstance(obj, abc.MutableSequence):
            return list(cls.build(item) for item in obj)
        return obj

In [6]:
f = FrozenJSON(data)
f.Schedule.speakers[0].name

'Robert Lefkowitz'

Instancja `FrozenJSON` zawiera prywatny atrybut instancji `__data` przechowywany pod nazwą `_FrozenJSON__data`. Próby odczytania atrybutów poprzez inne nazwy wyzwalają `__getattr__`. Metoda ta sprawdza czy słownik `self.__data` zawiera atrybut (nie klucz) o tej nazwie. To pozwala obsłużyć metody klasy `dict`. Jeśli nie ma atrybutu o podanej nazwie to używamy `name` jako klucza `self.__data` i przekazujemy odczytaną wartość do metody statycznej `build`. To pozwala na nawigowanie po zagnieżdżonych strukturach danych, ponieważ każde zagnieżdżone mapowanie jest konwertowane przez metodę `build` na kolejną instancję `FrozenJSON`.

In [7]:
student = FrozenJSON({"name": "John", "class": 25})
student.class

SyntaxError: invalid syntax (139714945.py, line 2)

In [8]:
import keyword


class FrozenJSON:
    """Fasada tylko do odczytu dla przeglądania obiektu podobnego do JSON
    przy używaniu notacji atrubytów."""

    def __init__(self, mapping):
        self.__data = dict()
        # Zamiana słów kluczowych na nazwy zakończone znakiem podkreślenia.
        for key, value in mapping.items():
            if keyword.iskeyword(key):
                key += "_"
            self.__data[key] = value

    def __getattr__(self, name):
        try:
            return getattr(self.__data, name)
        except AttributeError:
            return FrozenJSON.build(self.__data[name])

    def __dir__(self):
        return self.__data.keys()

    @classmethod
    def build(cls, obj):
        if isinstance(obj, abc.Mapping):
            return cls(obj)
        if isinstance(obj, abc.MutableSequence):
            return list(cls.build(item) for item in obj)
        return obj

In [9]:
student = FrozenJSON({"name": "John", "class": 25})
student.class_

25

### **Elastyczne Tworzenie Obiektów**

W Pythonie metodę `__init__` nazywamy metodą konstruktora, ale jest o tylko żargon. W Pythonie `__init__` przyjmuje `self` jako pierwszy argument, zatem obiekt już istnieje, gdy `__init__` jest wywoływany przez interpreter. Ponadto metoda ta nie może niczego zwrócić, zatem jest to raczej inicjator a nie konstruktor.

Gdy wywołujemy klasę w celu utworzenia instancji, metodą specjalną wywoływaną przez Pythona w celu skontruowania instancji klasy jest `__new__`. Jest to metoda klasy ale jest traktowana specjalnie, zatem nie jest stosowany dla niej dekorator `@classmethod`. Python przyjmuje instancję zwróconą przez `__new__` i przekazuje ją do `__init__` jako pierwszy argument. Jeśli zachodzi potrzeba, `__new__` może zwrócić instancję innej klasy. Gdy tak się zdarzy, interpreter nie wywołuje `__init__`.

In [10]:
# Pseduokod dla tworzenia obiektu
def make_object(class_, arg_):
    obj = class_.__new__(arg_)
    if isinstance(obj, class_):
        class_.__init__(obj, arg_)
    return obj

# To jest równoważne.
# x = Foo("bar")
# x = make_object(Foo, "bar")

In [11]:
class FrozenJSON:
    """Fasada tylko do odczytu dla przeglądania obiektu podobnego do JSON
    przy używaniu notacji atrubytów."""

    # Zamiana metody build na metodę __new__.
    # Pierwszym argumentem jest sama klasa a pozostałe są 
    # takie same jakie otrzymuje __init__.
    def __new__(cls, arg):
        if isinstance(arg, abc.Mapping):
            return super().__new__(cls)
        if isinstance(arg, abc.MutableSequence):
            return list(cls(item) for item in arg)
        return arg

    def __init__(self, mapping):
        self.__data = dict()
        # Zamiana słów kluczowych na nazwy zakończone znakiem podkreślenia.
        for key, value in mapping.items():
            if keyword.iskeyword(key):
                key += "_"
            self.__data[key] = value

    def __getattr__(self, name):
        try:
            return getattr(self.__data, name)
        except AttributeError:
            return FrozenJSON(self.__data[name])

    def __dir__(self):
        return self.__data.keys()

In [12]:
f = FrozenJSON(data)
f.Schedule.speakers[0].name

'Robert Lefkowitz'

Metoda `__new__` otrzymuje klasę jako pierwszy argument, ponieważ zwykle tworzony obiekt będzie instancją tej klasy. Tak więc w metodzie `FrozenJSON.__new__`, gdy wyrażenie `super().__new__(cls)` w efekcie wywołuje `object.__new__(FrozenJSON)`, instancją budowaną przez klasę `object` jest właściwie instancja `FrozenJSON`, tzn. atrybut `__class__` nowej instancji będzie zawierał odwołanie do `FrozenJSON`.

### **Buforowanie Właściwości Obliczanych**

#### **1. Sterowane Danymi Tworzenie Atrybutów**

In [24]:
PATH = "osconfeed-sample.json"


class Record:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

    def __repr__(self) -> str:
        return f"<{self.__class__.__name__} serial={self.serial!r}>"


def load(path=PATH):
    records = dict()
    with open(path) as f:
        data = json.load(f)
    for collection, raw_records in data["Schedule"].items():
        # Nazwa listy bez ostatniego znaku. Np 'speakers' -> 'speaker'
        record_type = collection.removesuffix("s")
        for raw_record in raw_records:
            key = f"{record_type}.{raw_record['serial']}"
            records[key] = Record(**raw_record)
    return records

In [25]:
records = load("osconfeed-sample.json")
records

{'conference.115': <Record serial=115>,
 'event.34505': <Record serial=34505>,
 'speaker.157509': <Record serial=157509>,
 'venue.1462': <Record serial=1462>}

In [26]:
speaker = records["speaker.157509"]
speaker.name, speaker.twitter

('Robert Lefkowitz', 'sharewaveteam')

#### **2. Właściwość Pobierająca Połączony Rekord**

Mając rekord `event`, odczytanie jesto właściwości `venue` ma zwrócić `Record`.

In [ ]:
import inspect


class Record:
    __index = None

    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

    def __repr__(self):
        return f"<{self.__class__.__name__} serial={self.serial!r}>"

    @staticmethod
    def fetch(key):
        if Record.__index is None:
            Record.__index = load()
        return Record.__index[key]
    
    
class Event(Record):
    def __repr__(self):
        try:
            return f"<{self.__class__.__name__} {self.name!r}>"
        except AttributeError:
            return super().__repr__()
        
    @property
    def venue(self):
        key = f"venue.{self.venue_serial}"
        return self.__class__.fetch(key)